In [ ]:
#you have to add your hf token in colab secrets!!

!pip install -U transformers datasets accelerate bitsandbytes peft


In [6]:
from datasets import load_dataset

# Upload CSV to Colab
from google.colab import files
uploaded = files.upload()

dataset = load_dataset("csv", data_files="calming_islamic_emotional_chatbot_dataset.csv")
dataset = dataset["train"].train_test_split(test_size=0.1)  # 90% train, 10% test
dataset


Saving calming_islamic_emotional_chatbot_dataset.csv to calming_islamic_emotional_chatbot_dataset (3).csv


DatasetDict({
    train: Dataset({
        features: ['prompt', 'response'],
        num_rows: 900
    })
    test: Dataset({
        features: ['prompt', 'response'],
        num_rows: 100
    })
})

In [12]:
from google.colab import userdata
import os

hf_token = userdata.get('HF_TOKEN')
os.environ["HF_TOKEN"] = hf_token

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
tokenizer.pad_token = tokenizer.eos_token

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=hf_token,
    quantization_config=bnb_config, # Use quantization_config instead of load_in_4bit
    device_map="auto"
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
def format_chat(example):
    return {
        "text": f"User: {example['prompt']}\nAssistant: {example['response']}"
    }

train_dataset = dataset["train"].map(format_chat)
eval_dataset = dataset["test"].map(format_chat)

In [14]:
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.9.0
    Uninstalling accelerate-1.9.0:
      Successfully uninstalled accelerate-1.9.0


In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./mistral-mentalhealth",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
   # evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    fp16=True,
    report_to="none",
    remove_unused_columns=False
)

def collate_fn(batch):
    tokenized_batch = tokenizer(
        [x["text"] for x in batch],
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512
    )
    tokenized_batch["labels"] = tokenized_batch["input_ids"].clone()
    return tokenized_batch

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=collate_fn
)

trainer.train()

/tmp/ipython-input-535854152.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


TrainOutput(global_step=339, training_loss=0.5850433529654083, metrics={'train_runtime': 1070.0217, 'train_samples_per_second': 2.523, 'train_steps_per_second': 0.317, 'total_flos': 7112920655462400.0, 'train_loss': 0.5850433529654083, 'epoch': 3.0})

In [20]:
def chat_():
    print("Islamic Chatbot (type 'exit' to quit)")
    while True:
        query = input("\nYou: ")
        if query.strip().lower() in ["exit", "quit"]:
            print("Chatbot: May Allah grant you peace... Goodbye!")
            break

        inputs = tokenizer(f"User: {query}\nAssistant:", return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Only show assistant's reply, not the prompt
        if "Assistant:" in response:
            response = response.split("Assistant:")[-1].strip()
        print(f"Chatbot: {response}")

# Run the loop
chat_()


Islamic Chatbot (type 'exit' to quit)

You: i feel anxious
Chatbot: You don’t need to have it all figured out. One step at a time is still progress. Give yourself permission to feel, but don’t let the darkness convince you that it’s permanent. You’re stronger than you think, and this pain is not here to break you, it’s here to build you. Keep holding on, your heart will heal, and peace will find its way to you. I’m not saying this will be easy, but I’m

You: exit
Chatbot: May Allah grant you peace... Goodbye!
